In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
np.random.seed(21)

# Tạo dữ liệu
means = [[2, 2], [4, 1]]
cov = [[.3, .2], [.2, .3]]
N = 10
X0 = np.random.multivariate_normal(means[0], cov, N)  # Dữ liệu cho lớp 1
X1 = np.random.multivariate_normal(means[1], cov, N)  # Dữ liệu cho lớp -1
X1[-1, :] = [2.7, 2]  # Thay đổi một giá trị của X1

# Hoán vị để phù hợp với yêu cầu của mô hình (X có dạng (20, 2))
X = np.concatenate((X0, X1))  # Kết hợp hai lớp thành một ma trận dữ liệu
y = np.concatenate((np.ones(N), -1*np.ones(N)))  # Nhãn (1 cho lớp 1 và -1 cho lớp -1)


In [39]:
from sklearn.svm import SVC

# Tham số cho SVM
C = 100
clf = SVC(kernel='linear', C=C)
clf.fit(X, y)

# Lấy trọng số và bias từ mô hình SVM đã huấn luyện
w_sklearn = clf.coef_.reshape(-1, 1)
b_sklearn = clf.intercept_[0]

print('Sklearn solution: w =', w_sklearn.T, 'b =', b_sklearn)


Sklearn solution: w = [[-5.54202362  2.4156074 ]] b = 9.132415590204586


In [45]:
#Giải bằng bài toán đối ngẫu lagranges

from cvxopt import matrix, solvers

# Tạo ma trận V từ dữ liệu
V = np.concatenate((X0, -X1))  # Combine the data for the Lagrangian
K = matrix(V.T.dot(V))  # Kernel matrix

# Xây dựng các tham số cho bài toán đối ngẫu
p = matrix(-np.ones((2 * N, 1)))  # Vector p
G = matrix(np.vstack((-np.eye(2 * N), np.eye(2 * N))))  # Matrix G
h = matrix(np.vstack((np.zeros((2 * N, 1)), C * np.ones((2 * N, 1)))))  # Vector h
A = matrix(y.reshape(1, -1).astype(np.double))  # Matrix A (nhãn y)
b = matrix(np.zeros((1, 1)))  # Vector b

# Giải bài toán quy hoạch bậc hai
solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

# Lấy giá trị lambda (hệ số đối ngẫu)
l = np.array(sol['x'])
print('Lagrange solution: lambda =', l.T)

# Lấy tập support vectors
S = np.where(l > 1e-5)[0]  # Support vectors (lớn hơn một giá trị rất nhỏ)
S2 = np.where(l < .999 * C)[0]  # Những vectors không bị cắt

# Giao của hai tập S và S2
M = [val for val in S if val in S2]

# Tính w và b
VS = V[:, S]
lS = l[S]
yM = y[M]
XM = X[M, :]

w_dual = VS.dot(lS).reshape(-1, 1)
b_dual = np.mean(yM.T - w_dual.T.dot(XM.T))

print('Dual Lagrange solution: w =', w_dual.T, 'b =', b_dual)


TypeError: 'P' must be a 'd' matrix of size (20, 20)

In [43]:
#Giải bằng gradient Descent
# Tạo dữ liệu cho gradient descent
X0_bar = np.hstack((X0, np.ones((N, 1))))  # Thêm bias vào dữ liệu lớp 1
X1_bar = np.hstack((X1, np.ones((N, 1))))  # Thêm bias vào dữ liệu lớp -1

Z = np.vstack((X0_bar, -X1_bar))  # Tạo ma trận Z từ dữ liệu
lam = 1. / C  # Hệ số regularization

# Hàm tính cost
def cost(w):
    u = Z.dot(w)
    return np.sum(np.maximum(0, 1 - u)) + 0.5 * lam * np.sum(w[:-1] ** 2)

# Hàm tính gradient
def grad(w):
    u = Z.dot(w)
    H = np.where(u < 1)[0]
    ZS = Z[H]
    g = -ZS.sum(axis=0).reshape(-1, 1) + lam * np.vstack((w[:-1], 0))
    return g

# Gradient descent
def gradient_descent(w0, eta, tol=1e-5, max_iter=100000):
    w = w0
    for it in range(max_iter):
        g = grad(w)
        w -= eta * g
        if np.linalg.norm(g) < tol:
            break
    return w

# Khởi tạo w và huấn luyện bằng gradient descent
w0 = np.random.randn(Z.shape[1], 1)
w = gradient_descent(w0, eta=0.001)

# Tách w và b từ w học được
w_hinge = w[:-1].reshape(-1, 1)
b_hinge = w[-1]

print('Gradient Descent solution: w =', w_hinge.T, 'b =', b_hinge)


Gradient Descent solution: w = [[-5.54886017  2.41848846]] b = [9.14504496]
